In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import spacy
from spacy.lang.de import German
from spacy import displacy
from spacy.lang.de.stop_words import STOP_WORDS
nlp = spacy.load('de_core_news_md')

+ 'Etat'
+ 'Inland'
+ 'International'
+ 'Kultur'
+ 'Panorama'
+ 'Sport'
+ 'Web'
+ 'Wirtschaft'
+ 'Wissenschaft'

Mehr Details zum Datenset unter: https://tblock.github.io/10kGNAD/


In [4]:
path = 'f:\\Data\\DataScienceUni\\40200\\BMW Bank Seminar\\Data Sets\\'

gnd = pd.read_csv(path+'10kGNAD_train.csv', header=None, sep=';', quotechar="'", names=['label', 'text'])
pd.set_option('display.max_colwidth', None)
print("Umfang des Datensets:",gnd.shape)
gnd_small = gnd[:5]

Umfang des Datensets: (9245, 2)


In [5]:
nlp(gnd.text[0])

21-Jähriger fällt wohl bis Saisonende aus. Wien – Rapid muss wohl bis Saisonende auf Offensivspieler Thomas Murg verzichten. Der im Winter aus Ried gekommene 21-Jährige erlitt beim 0:4-Heimdebakel gegen Admira Wacker Mödling am Samstag einen Teilriss des Innenbandes im linken Knie, wie eine Magnetresonanz-Untersuchung am Donnerstag ergab. Murg erhielt eine Schiene, muss aber nicht operiert werden. Dennoch steht ihm eine mehrwöchige Pause bevor.

In [10]:
def textcleaner(string):
    doc = nlp(string)
    string_clean = [w.lemma_ for w in doc if (not w.is_stop and not w.is_punct and not w.like_num)]
    return string_clean

In [9]:
gnd_small.text.apply(textcleaner2).apply(", ".join)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [11]:
gnd["text120"] = (gnd.text.str.split().str[:120]).apply(", ".join)
gnd["prepro"] = gnd.text.apply(textcleaner).apply(", ".join)
gnd.head()

label  \
0       Sport   
1      Kultur   
2         Web   
3  Wirtschaft   
4      Inland   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

Wir splitten wieder in Training (70%) und Test-Set (30%):

In [12]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(gnd, test_size=.3, stratify=gnd.label)
df_train.shape, df_test.shape

((6471, 4), (2774, 4))

In [13]:
df_train.label.value_counts(normalize=True).round(3)

Panorama         0.163
Web              0.163
International    0.147
Wirtschaft       0.137
Sport            0.117
Inland           0.099
Etat             0.065
Wissenschaft     0.056
Kultur           0.052
Name: label, dtype: float64

Wir importieren die notwendigen Classifier & Tools aus Scikit-Learn

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 

In [16]:
# classifier pipeline mit Support Vektor Machine als Classifier definieren
lsvc_classifier = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer(sublinear_tf=True)),                   
  ('clf', LinearSVC(dual=False, C=1.6, class_weight="balanced")),
])

# classifier mit vorgeschalteter Pipeline trainieren:
lsvc_classifier.fit(df_train.prepro, df_train.label)  # train the classifier

# trainierten classifier auf die Testdaten anwenden, um den Accuracy Score zu ermitteln:
predicted = lsvc_classifier.predict(df_test.prepro)  # predict the test set 
print(classification_report(predicted, df_test.label))

               precision    recall  f1-score   support

         Etat       0.75      0.90      0.82       150
       Inland       0.78      0.82      0.80       261
International       0.90      0.86      0.88       429
       Kultur       0.93      0.78      0.85       175
     Panorama       0.82      0.82      0.82       453
        Sport       0.97      0.98      0.97       323
          Web       0.86      0.91      0.89       430
   Wirtschaft       0.82      0.77      0.80       406
 Wissenschaft       0.81      0.86      0.83       147

     accuracy                           0.85      2774
    macro avg       0.85      0.85      0.85      2774
 weighted avg       0.86      0.85      0.85      2774



Zum Vergleich dasselbe nochmal mit einem anderen Classifier: "Logistische Regression"

In [ ]:
logReg_classifier = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer(sublinear_tf=True)),               
  ('clf', LogisticRegression(class_weight="balanced", solver='lbfgs', multi_class='auto'),),
])

logReg_classifier.fit(df_train['text'], df_train['label'])         # train the classifier
predicted = logReg_classifier.predict(df_test['text'])             # predict the test set 
acc = np.mean(predicted == df_test['label'])                       # calculate the accuracy

logReg_classifier.fit(df_train.text120, df_train.label) 
predicted = logReg_classifier.predict(df_test.text120)  

print(classification_report(predicted, df_test.label))

***
## Active Labeling mit 50 Trainingsdaten auf Basis k-Means (k=50)

In [20]:
from sklearn.cluster import KMeans
k = 50
kmeans = KMeans(n_clusters = k)

km = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer(sublinear_tf=True)),                   
  ('kmeans', KMeans(n_clusters = k)),
])


X_digits_dist = km.fit_transform(df_train.prepro)


# representative_digit_idx = np.argmin(X_digits_dist, axis=0)
# X_representative_digits = X_train[representative_digit_idx]
# X_representative_digits.shape

In [21]:
X_digits_dist

array([[1.02818835, 1.01961857, 1.0260788 , ..., 1.01991556, 1.04953802,
        1.04846197],
       [1.02736098, 1.01527377, 1.02277145, ..., 1.01937474, 1.05170887,
        1.03961289],
       [0.98891797, 1.01487573, 1.02429329, ..., 1.01628547, 1.02380328,
        1.02582261],
       ...,
       [1.01993713, 0.97458856, 0.94213429, ..., 1.01140067, 1.0430067 ,
        1.03704286],
       [1.0270394 , 1.01607825, 1.02346306, ..., 1.01768262, 1.05064973,
        1.04559519],
       [1.02264071, 1.00665581, 1.01464215, ..., 1.0141699 , 1.04771653,
        1.03629028]])

In [29]:
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
representative_digit_idx

array([4511, 1528, 3663, 5937, 5878, 3035, 4432, 6358, 2321, 4883, 5846,
       1040, 3109, 1020, 1312, 5435, 3453, 4162, 5064, 1653, 3370,  318,
        416, 4209,  578, 5512, 2995, 3010, 1443,  568, 4688, 3999, 2313,
        814,   13, 2481, 2254, 5495, 4949, 4181,  480, 4295, 2337, 1652,
       5667, 3867, 1833, 1244,  165, 5290], dtype=int64)

In [39]:
X_representative_digits = df_train.iloc[representative_digit_idx].prepro
X_representative_digits.shape

(50,)

In [44]:
y_representative_digits = df_train.iloc[representative_digit_idx].label
y_representative_digits.shape

(50,)

In [46]:
X_representative_digits.head()

5952    Deutscher, Verkehrsminister, Zahl, betroffen, Fahrzeug, stehen, fest, Seat-, Škoda-Modelle, betreffen, Wolfsburg, Volkswagen-Affäre, Manipulation, Abgasmessung, Dieselautos, ziehen, Kreis, Donnerstag, Fahrzeug, Europa, betreffen,  , Bekanntwerden, Skandal, USA, Volkswagen, mitteilen, weltweit, Million, Fahrzeug, betreffen, Europa, VW, Auto, Software, manipulieren, Test, deutlich, gesundheitsschädlich, Stickoxid, ausstoßen, tatsächlich, Straße, Konzernchef, Martin, Winterkorn, Affäre, Mittwoch, Rücktritt, Porsche-Chef, Matthias, Müller, Insider, zufolge, neu, Vorstandschef, Volkswagen, Aufsichtsrat, 62-Jährigen, Freitag, Nachfolger, Winterkorns, bestellen, Beratung, vertrauen, Person, Nachrichtenagentur, Reuter, Müller, Skandal, aufklären, Vertrauen, Volkswagen, zurückgewinnen, Volkswagen, äußern, Mitglied, Aufsichtsratspräsidiums, Donnerstagnachmittag, Müller, einigen, berichten, Handelsblatt, Frankfurter, Allgemeine, Zeitung, VW-Aufsichtsrat, tagen, Freitag, Personalie, beschl

In [49]:
df_train.label.sample(3)

2292              Web
2928       Wirtschaft
7055    International
Name: label, dtype: object

In [47]:
y_representative_digits.head()

5952    Wirtschaft
1323         Sport
1903      Panorama
4936           Web
677          Sport
Name: label, dtype: object

In [50]:
# classifier pipeline mit Support Vektor Machine als Classifier definieren
lsvc_classifier = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer(sublinear_tf=True)),                   
  ('clf', LinearSVC(dual=False, C=1.6, class_weight="balanced")),
])

# classifier mit vorgeschalteter Pipeline trainieren:
lsvc_classifier.fit(X_representative_digits, y_representative_digits)  # train the classifier

# trainierten classifier auf die Testdaten anwenden, um den Accuracy Score zu ermitteln:
predicted = lsvc_classifier.predict(df_test.prepro)  # predict the test set 
print(classification_report(predicted, df_test.label))

               precision    recall  f1-score   support

         Etat       0.09      0.74      0.17        23
       Inland       0.54      0.62      0.58       237
International       0.87      0.38      0.53       926
       Kultur       0.00      0.00      0.00         0
     Panorama       0.51      0.57      0.54       406
        Sport       0.96      0.52      0.67       599
          Web       0.62      0.81      0.70       344
   Wirtschaft       0.42      0.73      0.53       222
 Wissenschaft       0.10      0.94      0.19        17

     accuracy                           0.55      2774
    macro avg       0.46      0.59      0.43      2774
 weighted avg       0.73      0.55      0.58      2774



C:\Users\barth\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
km['kmeans'].labels_

array([11, 39, 29, ...,  2, 36,  7])

In [53]:
y_train_propagated = np.empty(len(df_train.prepro), dtype=np.int32)  # arbitrary filler data
for i in range(k):
    y_train_propagated[km["kmeans"].labels_==i] = y_representative_digits[i]
    # print(y_train_propagated[kmeans.labels_==i])


KeyError: 0

In [ ]:
y_train_propagated = np.empty(len(X_train), dtype=np.int32)  # arbitrary filler data
for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]
    print(y_train_propagated[kmeans.labels_==i])
plt.hist(y_train_propagated, density=True)   

# Parkplatz

In [ ]:
# os.chdir("F://Data/Data Science Uni/40200/BMW Bank Seminar/Data Sets")
# path = 'f:\\Data\\DataScienceUni\\40200\\BMW Bank Seminar\\Data Sets\\'